# Генерация данных по продажам а-ля qm

![](ord_prod_bd_scheme.png)

Входные данные: только таблица products: айди, название, категория, цена, затраты, частота покупок.

Создать список id продуктов с повторами в соответствии с частотой покупок - product_ids.

Функция create_new_users(last_user_id, number_of_new_users=250):
__генерим number_of_new_users *= (1 + 0.25 * снр)

В первый месяц создать 250± пользователей. 

В последующие месяцы в начале месяца:
- имитируем отток пользователей. Шафлим список юзеров. Убираем долю± клиентов (50%);
- добавлять так, чтобы было +250± человек ежемесячно. Тогда рост базы будет постепенно уменьшаться в процентах. Создаем список новых пользователей, шафлим и какие-то слева элементы добавляем еще раз, чтобы сымитировать разное время жизни клиентов. 0.04 - x7, 0.08 - x5, 0.11 - x3, 0.17 - x2. Получится, что долго живущие клиенты и покупать будут чаще, во всяком случае в начале своего жизненного пути.

Генерируем какое количество покупок в день случится. Число пользователей * среднее базовое число покупок в месяц / 30 ±.  
__Выбираем случайного пользователя из списка.  
____По остатку при делении id количество продуктов ± вычисляем. %11: <1 - 1, <3 - 2, <6 - 3, <9 - 4, <10 - 5, <11 - 6  
______Выбираем случайный продукт из списка и генерим случайное количество продуктов [1, 2, 3, 4, 5], [0.58, 0.28, 0.11, 0.02, 0.01]  

In [73]:
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [373]:
def create_new_users(users_id: list[int], last_user_id: int, number_of_new_users: int=8, volatility=0.3) -> int:
    number_of_new_users += int(number_of_new_users * volatility * random.normalvariate())
    if number_of_new_users < 1:
        number_of_new_users %= 100
    new_users_list = [*range(last_user_id + 1, last_user_id + number_of_new_users + 1)]
    random.shuffle(new_users_list)
    for share in [0.48, 0.33, 0.21, 0.12, 0.04, 0.02]:
        new_users_list.extend(new_users_list[:int(share * number_of_new_users)])
    users_id.extend(new_users_list)
    last_user_id += number_of_new_users
    return last_user_id, users_id

In [387]:
last_user_id, users_id = create_new_users([0], 0)
users_id, len(set(users_id))

([0, 4, 6, 2, 5, 3, 1, 4, 6, 4, 4], 7)

Создадим список айдишников товаров, в котором они будут повторяться по несколько раз в зависимости от частоты покупок этого товара:

In [62]:
df = pd.read_excel('products.xlsx')
df.head()

product_ids = []
for id, freq in zip(df['product_id'], df['freq']):
    product_ids.extend([id] * freq)

product_ids = np.array(product_ids)
product_ids[:10]

array([ 8, 25, 23, 23, 13, 13,  9, 12, 22, 22])